1. Мини-EDA анализа данных
Загрузим файлы data/clients.csv и data/bank_transactions.csv и соберём базовую статистику: количество записей, уникальных значений, распределение категорий, максимальное повторение и соотношение типов транзакций.

Описание:

Загружаем файлы и выводим количество строк, уникальных значений, типы данных и первые строки.
Анализируем распределение transaction_type (включая p2p и withdrawal).
Проверяем максимальное повторение по ключевым полям (client_id, merchant, ip_network), чтобы понять, какие связи доминируют.

In [ ]:
import pandas as pd

# Загружаем данные
clients = pd.read_csv('data/clients.csv')
transactions = pd.read_csv('data/bank_transactions.csv')

# Базовая статистика по clients
print("=== Статистика по clients.csv ===")
print(f"Количество строк: {len(clients)}")
print(f"Количество уникальных клиентов (client_id): {clients['client_id'].nunique()}")
print("\nСтолбцы и их типы:")
print(clients.dtypes)
print("\nПервые 5 строк:")
display(clients.head())
print("\nУникальные значения по столбцам:")
for column in clients.columns:
    print(f"{column}: {clients[column].nunique()} уникальных значений")

# Базовая статистика по transactions
print("\n=== Статистика по bank_transactions.csv ===")
print(f"Количество строк: {len(transactions)}")
print(f"Количество уникальных транзакций (transaction_id): {transactions['transaction_id'].nunique()}")
print("\nСтолбцы и их типы:")
print(transactions.dtypes)
print("\nПервые 5 строк:")
display(transactions.head())
print("\nУникальные значения по столбцам:")
for column in transactions.columns:
    print(f"{column}: {transactions[column].nunique()} уникальных значений")

# Распределение типов транзакций
print("\n=== Распределение типов транзакций ===")
transaction_type_counts = transactions['transaction_type'].value_counts()
print(transaction_type_counts)
print(f"Доля P2P: {transaction_type_counts.get('p2p', 0) / len(transactions) * 100:.2f}%")
print(f"Доля withdrawal: {transaction_type_counts.get('withdrawal', 0) / len(transactions) * 100:.2f}%")
print(f"Доля deposit: {transaction_type_counts.get('deposit', 0) / len(transactions) * 100:.2f}%")

# Максимальное повторение по ключевым полям
print("\n=== Максимальное повторение ===")
print(f"Максимальное повторение client_id: {transactions['client_id'].value_counts().max()}")
print(f"Максимальное повторение merchant: {transactions['merchant'].value_counts().dropna().max()}")
print(f"Максимальное повторение ip_address: {transactions['ip_address'].value_counts().dropna().max()}")

Ячейка 2: Подробный анализ связей
Проверим, как часто клиенты пересекаются по различным полям, чтобы понять потенциальные рёбра.

Описание:

Проверяем распределение клиентов по мерчантам, чтобы выявить крупных игроков.
Анализируем P2P и withdrawal, включая уникальные recipient_id_hash и client_id.
Проверяем, насколько session_id может генерировать рёбра.


In [ ]:
# Анализ пересечений по мерчантам
merchant_pairs = transactions[transactions['merchant'].notna()].groupby('merchant')['client_id'].nunique().reset_index(name='client_count')
print("\n=== Количество клиентов по мерчантам ===")
print(merchant_pairs.sort_values('client_count', ascending=False).head(10))

# Анализ P2P и withdrawal
p2p_data = transactions[transactions['transaction_type'] == 'p2p']
withdrawal_data = transactions[transactions['transaction_type'] == 'withdrawal']
print(f"\nКоличество P2P-транзакций: {len(p2p_data)}")
print(f"Количество withdrawal-транзакций: {len(withdrawal_data)}")
print(f"Уникальных recipient_id в P2P: {p2p_data['recipient_id'].nunique()}")
print(f"Уникальных client_id в withdrawal: {withdrawal_data['client_id'].nunique()}")

# Анализ session_id
session_counts = transactions.groupby('session_id')['client_id'].nunique().reset_index(name='client_count')
session_multiple = session_counts[session_counts['client_count'] > 1]
print(f"\nКоличество session_id, используемых несколькими клиентами: {len(session_multiple)}")
print("Топ-5 session_id по количеству клиентов:")
print(session_multiple.sort_values('client_count', ascending=False).head())

# Анализ регионов и стран
print(f"\nУникальных регионов: {transactions['region'].nunique()}")
print(f"Уникальных стран: {transactions['country_code'].nunique()}")

Описание:

Строим граф с нуля на основе bank_transactions.csv.
Добавляем рёбра по мерчантам (с фильтром до 100 клиентов), P2P и session_id.
Используем Label Propagation для кластеризации.

In [ ]:
import networkx as nx
import pandas as pd
from datetime import timedelta

# Загружаем данные
transactions = pd.read_csv('data/bank_transactions.csv')
transactions['datetime'] = pd.to_datetime(transactions['datetime'])

# Создаём граф
G = nx.Graph()

# Добавляем узлы
clients_set = set(transactions['client_id'])
for client in clients_set:
    G.add_node(client)

# Создаём рёбра
edges = []

# Рёбра по общим мерчантам (только соседние по времени в пределах 1 дня)
merchant_groups = transactions[transactions['merchant'].notna()].sort_values('datetime')
for merchant, group in merchant_groups.groupby('merchant'):
    if len(group) <= 100:  # Фильтр по количеству клиентов
        for i in range(len(group) - 1):
            curr_client = group.iloc[i]['client_id']
            next_client = group.iloc[i + 1]['client_id']
            curr_time = group.iloc[i]['datetime']
            next_time = group.iloc[i + 1]['datetime']
            if next_time - curr_time <= timedelta(days=1) and curr_client != next_client:
                edges.append((curr_client, next_client, {'weight': 1.0, 'relationship': 'merchant_shared'}))

# Рёбра по P2P
p2p_edges = transactions[transactions['transaction_type'] == 'p2p']
for _, row in p2p_edges.iterrows():
    if pd.notna(row['recipient_id']) and row['client_id'] < row['recipient_id']:
        edges.append((row['client_id'], row['recipient_id'], {'weight': 10.0, 'relationship': 'p2p'}))

# Рёбра по session_id (только если более 5 клиентов)
session_groups = transactions.groupby('session_id')['client_id'].apply(list).reset_index(name='clients')
for _, row in session_groups.iterrows():
    clients_list = row['clients']
    if len(clients_list) > 5:
        for i in range(len(clients_list)):
            for j in range(i + 1, len(clients_list)):
                edges.append((clients_list[i], clients_list[j], {'weight': 0.5, 'relationship': 'session_shared'}))

# Рёбра по withdrawal (клиенты в одном регионе и дате)
withdrawal_groups = transactions[transactions['transaction_type'] == 'withdrawal'].groupby(['region', 'datetime'])['client_id'].apply(list).reset_index(name='clients')
for _, row in withdrawal_groups.iterrows():
    clients_list = row['clients']
    if len(clients_list) > 1:
        for i in range(len(clients_list)):
            for j in range(i + 1, len(clients_list)):
                edges.append((clients_list[i], clients_list[j], {'weight': 2.0, 'relationship': 'withdrawal_shared'}))

# Добавляем рёбра в граф
G.add_edges_from(edges)

print(f"Создан граф с {G.number_of_nodes()} узлами и {G.number_of_edges()} рёбрами")

# Кластеризация с Label Propagation
from networkx.algorithms.community import label_propagation_communities
communities = list(label_propagation_communities(G))
print(f"Найдено {len(communities)} сообществ")

# Статистика
community_sizes = [len(c) for c in communities]
community_sizes.sort(reverse=True)
print(f"Топ-5 размеров сообществ: {community_sizes[:5]}")
print(f"Доля крупнейшего сообщества: {community_sizes[0] / sum(community_sizes) * 100:.2f}%")

# Установка и использование Louvain
try:
    import community as community_louvain
    partition = community_louvain.best_partition(G)
    community_sizes_louvain = {comm: list(partition.values()).count(comm) for comm in set(partition.values())}
    community_sizes_louvain = dict(sorted(community_sizes_louvain.items(), key=lambda x: x[1], reverse=True))
    print(f"\nТоп-5 размеров сообществ (Louvain): {list(community_sizes_louvain.values())[:5]}")
    print(f"Доля крупнейшего сообщества (Louvain): {list(community_sizes_louvain.values())[0] / sum(community_sizes_louvain.values()) * 100:.2f}%")
except ImportError:
    print("Библиотека community не установлена. Установите её с помощью 'pip install python-louvain'.")

In [ ]:
# VISUAL_ANALYTICS.PY
# -------------------
import networkx as nx, pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from networkx.algorithms.community import label_propagation_communities
from pylab import rcParams
from community import modularity         # из python-louvain
import random, warnings; warnings.filterwarnings("ignore")

# ---- 1. сообщества Louvain (уже считаны ранее) --------------
louvain = nx.get_node_attributes(G, 'louvain_comm')      # из прошлого скрипта

# если не считано – считаем
if not louvain:
    import community as community_louvain
    louvain = community_louvain.best_partition(G, weight='weight')
    nx.set_node_attributes(G, louvain, 'louvain_comm')

# ---- 2. Label Propagation -----------------------------------
if 'lp_comm' not in nx.get_node_attributes(G, 'lp_comm'):
    lp_raw = list(label_propagation_communities(G))
    lp_map = {}
    for i, comm in enumerate(lp_raw):
        for node in comm:
            lp_map[node] = i
    nx.set_node_attributes(G, lp_map, 'lp_comm')
else:
    lp_map = nx.get_node_attributes(G, 'lp_comm')

# ---- 3. Метрики графа ---------------------------------------
n  = G.number_of_nodes();  m = G.number_of_edges()
density      = nx.density(G)
avg_deg      = round(2*m/n, 2)
avg_clust    = nx.average_clustering(G, weight='weight')
largest_cc   = max(nx.connected_components(G), key=len)
diameter_lcc = nx.diameter(G.subgraph(largest_cc))
lou_mod      = modularity(louvain, G, weight='weight')
lp_mod       = modularity(lp_map, G, weight='weight')

metrics = pd.Series({
    'nodes'              : n,
    'edges'              : m,
    'density'            : density,
    'avg_weighted_degree': avg_deg,
    'avg_clustering'     : avg_clust,
    'diameter_LCC'       : diameter_lcc,
    'modularity_louvain' : lou_mod,
    'modularity_lp'      : lp_mod
})
print(metrics.to_markdown())

# ---- 4. Pie-chart размеров сообществ (Louvain) --------------
sizes = pd.Series(louvain).value_counts().sort_values(ascending=False)
top10 = sizes.head(10)
fig, ax = plt.subplots(figsize=(7,7))
ax.pie(top10, labels=[f'C{c}' for c in top10.index],
       autopct='%1.1f%%', pctdistance=0.8, startangle=90)
ax.set_title('Top-10 community share (Louvain)')
plt.show()

# ---- 5. Гистограмма лог-взвешенной степени ------------------
deg = pd.Series(dict(G.degree(weight='weight')))
sns.displot(np.log1p(deg), kde=True, height=4, aspect=1.6)
plt.title('Histogram of log-weighted degrees')
plt.xlabel('log1p(weighted degree)');  plt.ylabel('freq')
plt.show()

# ---- 6. Функция выборочной визуализации графа ---------------
def draw_subgraph(comm_attr, comm_name, k=700):
    # берём k случайных узлов + их соседей
    base_nodes = random.sample(list(G.nodes), min(k, len(G)))
    nbrs = set(base_nodes)
    for n in base_nodes:
        nbrs.update(G[n])
    SG = G.subgraph(nbrs).copy()

    # выбираем цвет по сообществу
    comms = nx.get_node_attributes(G, comm_attr)
    colors = [comms.get(v, -1) for v in SG.nodes()]
    cmap = plt.cm.get_cmap('tab20')

    pos = nx.spring_layout(SG, seed=42, k=0.2)
    rcParams['figure.figsize'] = 10,8
    plt.figure()
    nx.draw_networkx_nodes(SG, pos,
                           node_size=40,
                           node_color=colors,
                           cmap=cmap,
                           alpha=0.9)
    nx.draw_networkx_edges(SG, pos,
                           width=0.3,
                           alpha=0.4)
    plt.title(f"Sampled graph coloured by {comm_name} communities")
    plt.axis('off')
    plt.show()

draw_subgraph('louvain_comm', 'Louvain')
draw_subgraph('lp_comm',      'Label Propagation')
